# Import Library

In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Data

In [77]:
df = pd.read_csv('goodreads_data.csv')
df.head()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


# EDA

In [78]:
df.shape

(10000, 8)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [80]:
df.isnull().sum()

Unnamed: 0      0
Book            0
Author          0
Description    77
Genres          0
Avg_Rating      0
Num_Ratings     0
URL             0
dtype: int64

In [81]:
df['Genres'].unique()

array(["['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']",
       "['Fantasy', 'Fiction', 'Young Adult', 'Magic', 'Childrens', 'Middle Grade', 'Classics']",
       "['Classics', 'Fiction', 'Romance', 'Historical Fiction', 'Literature', 'Historical', 'Audiobook']",
       ...,
       "['Dystopia', 'Science Fiction', 'Post Apocalyptic', 'Paranormal', 'Fantasy']",
       "['Fiction', 'Horror', 'Dystopia', 'Coming Of Age']",
       "['New Adult', 'Romance', 'Contemporary Romance', 'Contemporary']"],
      dtype=object)

In [90]:
df['Avg_Rating'].value_counts()

Avg_Rating
4.00    202
4.12    158
4.18    154
4.09    153
4.14    152
       ... 
3.16      1
3.04      1
2.97      1
2.03      1
3.05      1
Name: count, Length: 211, dtype: int64

**Insight**
- Dapat diketahui pada dataset tersebut memiliki 10.000 baris dan 8 kolom.
- Missing value terdeteksi sebanyak 77 di kolom `Description`, namun kolom yang lainnya lengkap.
- **Genre**: Variasi besar, sebagian besar genre adalah **Fiction** dan **Fantasy**.
- **Rating**: Mayoritas buku memiliki rating tinggi, dengan rating **4.00** yang paling sering muncul.
- **Kolom `Num_Ratings`**: Tipe data `object` perlu dikonversi menjadi numerik setelah menghapus tanda koma.
- Tahap selanjutnya perlu pembersihan pada kolom `Genres` dan `Description`, serta konversi tipe data di `Num_Ratings` untuk analisis lebih lanjut.


# Data Preparation

In [83]:
# Menghapus kolom yang tidak diperlukan
df_cleaned = df.drop(columns=['Unnamed: 0', 'URL'])

In [84]:
# Menghapus koma pada kolom 'Num_Ratings'
df_cleaned['Num_Ratings'] = df_cleaned['Num_Ratings'].str.replace(',', '')

# Mengonversi kolom 'Num_Ratings' menjadi float64
df_cleaned['Num_Ratings'] = pd.to_numeric(df_cleaned['Num_Ratings'], errors='coerce')

In [85]:
# Mengisi nilai kosong di kolom 'Description' dengan string kosong
df_cleaned['Description'].fillna('No Description', inplace=True)

C:\Users\dumbw\AppData\Local\Temp\ipykernel_8184\3979502448.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Description'].fillna('No Description', inplace=True)


In [86]:
# Menggabungkan kolom 'Genres', 'Description', 'Author', dan 'Book' menjadi satu kolom teks
df_cleaned['combined'] = df_cleaned['Genres'] + ' ' + df_cleaned['Description'] + ' ' + df_cleaned['Author'] + ' ' + df_cleaned['Book']

# Melihat data yang telah digabungkan
print(df_cleaned[['Book', 'combined']].head())

                                                Book  \
0                              To Kill a Mockingbird   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                                Pride and Prejudice   
3                          The Diary of a Young Girl   
4                                        Animal Farm   

                                            combined  
0  ['Classics', 'Fiction', 'Historical Fiction', ...  
1  ['Fantasy', 'Fiction', 'Young Adult', 'Magic',...  
2  ['Classics', 'Fiction', 'Romance', 'Historical...  
3  ['Classics', 'Nonfiction', 'History', 'Biograp...  
4  ['Classics', 'Fiction', 'Dystopia', 'Fantasy',...  


Berikut adalah insight dari tahap **Data Preparation**:

1. **Penghapusan Kolom yang Tidak Diperlukan**: Kolom `Unnamed: 0` dan `URL` dihapus karena tidak relevan untuk analisis lebih lanjut.

2. **Pembersihan Data `Num_Ratings`**: Tanda koma (`,`) dihapus dari kolom `Num_Ratings`, kemudian kolom ini dikonversi ke tipe data numerik (`float64`) untuk analisis yang lebih akurat.

3. **Penanganan Data Kosong**: Nilai kosong pada kolom `Description` diisi dengan string `'No Description'` untuk menghindari masalah saat pengolahan data.

4. **Penggabungan Kolom Teks**: Kolom-kolom `Genres`, `Description`, `Author`, dan `Book` digabungkan menjadi satu kolom baru `combined`. Hal ini memungkinkan pemrosesan teks yang lebih efektif untuk analisis dan pencarian kemiripan buku.

5. **Output Data Gabungan**: Kolom `combined` kini berisi informasi lengkap tentang genre, deskripsi, penulis, dan nama buku, yang dapat digunakan untuk perhitungan kemiripan teks atau rekomendasi buku.

Secara keseluruhan, data telah dipersiapkan dengan baik untuk analisis lebih lanjut seperti pemrosesan teks dan pencarian kemiripan antar buku.


# Modeling

In [87]:
# Menggunakan TfidfVectorizer untuk menghitung representasi TF-IDF dari teks yang telah digabungkan
tfidf = TfidfVectorizer(stop_words='english')

# Menghitung matriks TF-IDF
tfidf_matrix = tfidf.fit_transform(df_cleaned['combined'])

# Menghitung cosine similarity antar buku menggunakan TF-IDF matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Menampilkan kemiripan antara 5 buku pertama
print(cosine_sim[:5, :5])


[[1.         0.01562193 0.03846026 0.03965998 0.03384114]
 [0.01562193 1.         0.00418993 0.00601878 0.01892882]
 [0.03846026 0.00418993 1.         0.01120675 0.03671264]
 [0.03965998 0.00601878 0.01120675 1.         0.00728016]
 [0.03384114 0.01892882 0.03671264 0.00728016 1.        ]]


In [88]:
def recommend_books(query, cosine_sim=cosine_sim, top_n=5):
    # Mengubah kata kunci (query) menjadi representasi TF-IDF
    query_tfidf = tfidf.transform([query])
    
    # Menghitung kemiripan cosine antara query dan buku-buku
    cosine_sim_query = cosine_similarity(query_tfidf, tfidf_matrix)
    
    # Mengurutkan artikel berdasarkan skor kemiripan secara menurun
    sim_scores = list(enumerate(cosine_sim_query[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Mengambil top_n buku teratas (mengabaikan buku itu sendiri)
    sim_scores = sim_scores[0:top_n]
    
    # Mendapatkan indeks buku yang direkomendasikan
    book_indices = [i[0] for i in sim_scores]
    
    # Mengembalikan buku yang direkomendasikan berdasarkan genre, rating, dan jumlah rating tertinggi
    recommended_books = df_cleaned[['Book', 'Author', 'Genres', 'Avg_Rating', 'Num_Ratings']].iloc[book_indices]
    
    # Menyortir berdasarkan jumlah rating tertinggi
    recommended_books['Num_Ratings'] = recommended_books['Num_Ratings'].apply(pd.to_numeric, errors='coerce')
    recommended_books = recommended_books.sort_values(by='Num_Ratings', ascending=False)
    
    return recommended_books.head(top_n)


In [89]:
recommended_books = recommend_books('history')
print("Top 10 recommended books based on the query:")
print(recommended_books)

Top 10 recommended books based on the query:
                                                   Book              Author  \
234             A People's History of the United States         Howard Zinn   
1995           At Home: A Short History of Private Life         Bill Bryson   
8359  The Discoverers: A History of Man's Search to ...  Daniel J. Boorstin   
8577  The History of the Ancient World: From the Ear...    Susan Wise Bauer   
3606           Shit my History Teacher DID NOT tell me!        Karl Wiggins   

                                                 Genres  Avg_Rating  \
234   ['History', 'Nonfiction', 'Politics', 'America...        4.07   
1995  ['Nonfiction', 'History', 'Audiobook', 'Humor'...        3.98   
8359  ['History', 'Nonfiction', 'Science', 'Philosop...        4.12   
8577  ['History', 'Nonfiction', 'Ancient History', '...        4.11   
3606  ['Nonfiction', 'History', 'Humor', 'Comedy', '...        4.06   

      Num_Ratings  
234        232434  
1995        9

Berikut adalah insight dari tahap **Modeling**:

1. **Penerapan TF-IDF**: TF-IDF digunakan untuk menghitung representasi teks dari kolom `combined`. Ini memungkinkan analisis berbasis teks yang lebih efektif dengan mengurangi pengaruh kata-kata umum dan memberikan bobot lebih pada kata-kata yang lebih unik dalam setiap buku.

2. **Perhitungan Cosine Similarity**: Matriks **cosine similarity** dihitung antar buku berdasarkan representasi TF-IDF mereka. Nilai dalam matriks menunjukkan seberapa mirip setiap pasangan buku berdasarkan gabungan teks mereka. Hasil awal menunjukkan nilai kemiripan antara buku-buku yang sangat rendah, seperti antara buku pertama dan kedua yang hanya memiliki kemiripan 0.0156.

3. **Rekomendasi Buku**: Fungsi `recommend_books` digunakan untuk mengembalikan buku-buku yang paling mirip berdasarkan query yang diberikan (misalnya, "history"). Buku-buku yang direkomendasikan kemudian disaring berdasarkan jumlah rating yang diterima, memberikan prioritas pada buku yang lebih banyak di-rating.

4. **Output Rekomendasi**: Dengan menggunakan kata kunci seperti "history", rekomendasi buku dihasilkan berdasarkan kemiripan teks dengan buku-buku lainnya dalam dataset. Hasil tersebut memberikan daftar buku dengan genre terkait, rating, dan jumlah rating yang relevan.

Secara keseluruhan, tahap ini berhasil mempersiapkan model rekomendasi berbasis teks menggunakan TF-IDF dan cosine similarity, yang siap untuk memberikan rekomendasi buku berdasarkan kata kunci yang diberikan.